In [2]:
from __future__ import division, print_function

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

In [4]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [27]:
benchmarks = pd.read_csv('data/benchmars_linears_trzmiel.csv')
#benchmarks = benchmarks.ix[benchmarks.estimator != 'Links(labels+links)']

In [28]:
benchmarks.head()

,cv_splits,rs_test_size,rs_iters,cv_test_size,rs_splits,cv_random_state,dataset,estimator,percent_labels,percent_links,percent_unlabeled,cv_split,alpha,beta,cv_score,gamma,test_score,delta,C
0,10.0,0.2,100.0,0.2,5.0,42.0,diabetes_scale,Links,0.1,0.1,0.0,0.0,46.910631,546.033741,0.784615,0.01000,0.740260,NaN,NaN
1,10.0,0.2,100.0,0.2,5.0,42.0,diabetes_scale,Links,0.1,0.1,0.1,0.0,98.108611,838.277879,0.861538,0.00001,0.740260,71.648611,NaN
2,10.0,0.2,100.0,0.2,5.0,42.0,diabetes_scale,Links,0.1,0.1,0.0,4.0,223.521513,675.757103,0.692308,100.00000,0.740260,NaN,NaN
3,10.0,0.2,100.0,0.2,5.0,42.0,diabetes_scale,Links,0.1,0.1,0.0,9.0,46.910631,546.033741,0.784615,0.01000,0.740260,NaN,NaN
4,10.0,0.2,100.0,0.2,5.0,42.0,diabetes_scale,Links,0.2,0.2,0.0,4.0,94.795387,1103.436826,0.768000,0.01000,0.720779,NaN,NaN


In [29]:
%matplotlib tk

In [30]:
def plot_benchmarks_subset(benchmarks_subset):
    fig, axs= plt.subplots(ncols=len(benchmarks_subset.percent_labels.unique()), nrows=len(benchmarks_subset.dataset.unique()))

    for i_ds, (ds_name, ds_df) in enumerate(benchmarks_subset.groupby('dataset')):

        for i_percent, (p_labels, percent_subset) in enumerate(ds_df.groupby('percent_labels')):
            #print(p_labels, p_links)
            #print(percent_subset.estimator.unique())
            ax = axs[i_ds, i_percent]
            if i_percent == 0:
                ax.set_title(ds_name)
            ax.set_xlim(0, 0.5)
            #ax.set_xlabel('% unlabeled')
            #ax.set_ylim(0.55, 1)
            ax.set_ylim(percent_subset.test_score.min(), percent_subset.test_score.max())
            #ax.set_ylabel('accuracy')

            svm_scores = percent_subset.ix[percent_subset.estimator == 'SVM']
            npklr_scores = percent_subset.ix[percent_subset.estimator == 'NPKLR']
            links_scores = percent_subset.ix[percent_subset.estimator == 'Links']
            links_score_by_pun = links_scores['test_score'].groupby(links_scores['percent_unlabeled']).agg('mean')
            #assert svm_scores.
            ax.plot([0, 0.5], [svm_scores.test_score.mean()]*2, 'g--', label='SVM')
            ax.plot([0, 0.5], [npklr_scores.test_score.mean()]*2, 'y--', label='NPKLR')
            ax.plot(links_score_by_pun.index.values, links_score_by_pun.values, 'r', label='Links')
        #break
    fig.subplots_adjust(left=0.05, bottom=0.05, right=0.95, top=0.95,
                    wspace=0.07, hspace=0.05)

In [25]:
benchmarks_subset = benchmarks.ix[benchmarks.dataset.isin(benchmarks.dataset.unique()[:3])]
plot_benchmarks_subset(benchmarks_subset)

In [26]:
benchmarks_subset = benchmarks.ix[benchmarks.dataset.isin(benchmarks.dataset.unique()[3:])]
plot_benchmarks_subset(benchmarks_subset)

In [31]:
plot_benchmarks_subset(benchmarks)

In [51]:
benchmarks.dataset.unique()

array(['diabetes_scale', 'breast-cancer_scale', 'australian_scale',
       'ionosphere_scale', 'german.numer_scale', 'heart_scale',
       'liver-disorders_scale'], dtype=object)

In [32]:
lapsvm = pd.read_csv('data/remote/lapsvm_trzmiel.csv')
lapsvm.ix[pd.isnull(lapsvm.estimator), 'estimator'] = 'Links'

In [33]:
lapsvm.head()

,dataset,percent_labels,percent_links,percent_unlabeled,cv_split,rs_iters,rs_splits,rs_test_size,cv_score,test_score,beta,delta,alpha,gamma,estimator,gamma_I,gamma_A
0,diabetes_scale,0.1,0.1,0.1,4.0,100.0,5.0,0.2,0.876923,0.694805,627.036243,62.386288,241.334168,10.0,Links,NaN,NaN
1,diabetes_scale,0.1,0.1,0.1,6.0,100.0,5.0,0.2,0.876923,0.616883,576.105660,14.394807,16.799070,10.0,Links,NaN,NaN
2,diabetes_scale,0.1,0.1,0.1,3.0,100.0,5.0,0.2,0.876923,0.740260,627.036243,62.386288,241.334168,10.0,Links,NaN,NaN
3,diabetes_scale,0.1,0.1,0.1,5.0,100.0,5.0,0.2,0.830769,0.636364,48.904148,4.068718,138.640532,100.0,Links,NaN,NaN
4,diabetes_scale,0.1,0.1,0.1,0.0,100.0,5.0,0.2,0.830769,0.766234,473.126960,0.355701,817.840352,10.0,Links,NaN,NaN


In [36]:
def plot_lapsvm_subset(lapsvm_subset):
    fig, axs= plt.subplots(ncols=len(lapsvm_subset.percent_labels.unique()), nrows=len(lapsvm_subset.dataset.unique()))

    for i_ds, (ds_name, ds_df) in enumerate(lapsvm_subset.groupby('dataset')):
        for i_percent, (p_labels, percent_subset) in enumerate(ds_df.groupby('percent_labels')):
            
            #print(percent_subset.estimator.unique())
            ax = axs[i_ds, i_percent]
            if i_percent == 0:
                ax.set_title(ds_name)
            ax.set_xlim(0, 0.5)
            #ax.set_xlabel('% unlabeled')
            # ax.set_ylim(0.55, 1)
            ax.set_ylim(percent_subset.test_score.min(), percent_subset.test_score.max())
            #ax.set_ylabel('accuracy')

            lapsvm_scores = percent_subset.ix[percent_subset.estimator == 'LapSVM']
            
            links_scores = percent_subset.ix[percent_subset.estimator == 'Links']
            links_score_by_plinks = links_scores['test_score'].groupby(links_scores['percent_links']).agg('mean')
            #assert svm_scores.
            ax.plot([0, 0.5], [lapsvm_scores.test_score.mean()]*2, 'g--', label='LapSVM')
            ax.plot(links_score_by_plinks.index.values, links_score_by_plinks.values, 'r', label='Links')
        #break
    #fig.tight_layout(pad=0.11, w_pad=0.1, h_pad=0.05)
    fig.subplots_adjust(left=0.05, bottom=0.05, right=0.95, top=0.95,
                    wspace=0.07, hspace=0.05)

In [37]:
plot_lapsvm_subset(lapsvm)

In [67]:
lapsvm.percent_links.unique()

array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0. ])